第十六章协程

16.2 用作协程的生成器的基本行为

In [11]:
def simple_coroutine():
    print('-> cotoutine started')
    x=yield 
    print('-> coroutine received:  ',x)
my_coro=simple_coroutine()
my_coro

<generator object simple_coroutine at 0x0000026A41F999E8>

In [12]:
next(my_coro)

-> cotoutine started


In [13]:
my_coro.send(42)#协程处于暂停状态时才能调用send方法

-> coroutine received:   42


StopIteration: 

In [16]:
my_coro=simple_coroutine()
my_coro.send(42)

TypeError: can't send non-None value to a just-started generator

In [17]:
#产出两个值得协程
def simple_coro2(a):
    print('->get started:a= ',a)
    b=yield a
    print('-> received: b= ',b)
    c=yield a+b
    print('-> received:c= ',c)
my_coro2=simple_coro2(14)

In [18]:
from  inspect import getgeneratorstate
getgeneratorstate(my_coro2)

'GEN_CREATED'

In [19]:
next(my_coro2)

->get started:a=  14


14

In [20]:
getgeneratorstate(my_coro2)

'GEN_SUSPENDED'

In [21]:
my_coro2.send(28)

-> received: b=  28


42

In [22]:
getgeneratorstate(my_coro2)

'GEN_SUSPENDED'

In [23]:
my_coro2.send(99)

-> received:c=  99


StopIteration: 

In [24]:
getgeneratorstate(my_coro2)

'GEN_CLOSED'

16.3 使用协程计算移动平均值

In [25]:
def averager():
    total=0.0
    count=0
    average=None
    while True:
        term = yield average
        total+=term
        count+=1
        average=total/count

In [30]:
coro_avg=averager()
next(coro_avg)

In [31]:
coro_avg.send(10)

10.0

In [32]:
coro_avg.send(30)

20.0

16.4 预激协程的装饰器

In [4]:
#如果不预激，那么协程没什么用，为了简化协程的用法，有时会使用一个预激装饰器
from functools import wraps
def coroutine(func):
    '''装饰器，向前执行到第一个yield表达式，预激func'''
    @wraps(func)
    def primer(*args,**kwargs):
        gen=func(*args,**kwargs)
        next(gen)
        return gen
    return primer 

In [5]:
@coroutine
def averager():
    total=0.0
    count=0
    average=None
    while True:
        term = yield average
        total+=term
        count+=1
        average=total/count   

In [7]:
coro_avg=averager()
from inspect import getgeneratorstate
getgeneratorstate(coro_avg)#生成器可直接使用send了

'GEN_SUSPENDED'

16.5 终止协程和异常处理

In [8]:
import sys
sys.path.insert(0,r'D:\WORKSPACE2\python35\python8.25\fluent_python')

In [9]:
coro_avg=averager()
coro_avg.send(40)

40.0

In [10]:
coro_avg.send(50)

45.0

In [11]:
coro_avg.send('spam')

TypeError: unsupported operand type(s) for +=: 'float' and 'str'

In [12]:
coro_avg.send(60)#由于在协程内没有处理异常，协程会终止。

StopIteration: 

generator.throw()和generator.close()

In [13]:
class DemoException(Exception):
    '''为演示定义的异常类型'''

def demo_exc_handling():
    print('->continue started')
    while True:
        try:
            x=yield 
        except DemoException:
            print('***DemoException handled.Continue ...')
        else:
            print('->coroutine received: {!r}'.format(x))
    raise RuntimeError('This line should never run')#这行代码永远不会执行

In [14]:
exc_coro=demo_exc_handling()
next(exc_coro)

->continue started


In [15]:
exc_coro.send(11)

->coroutine received: 11


In [16]:
exc_coro.send(22)

->coroutine received: 22


In [17]:
exc_coro.close()

In [19]:
getgeneratorstate(exc_coro)

'GEN_CLOSED'

In [20]:
#把DemoException传入demo_exc_handling不会导致协程终止
exc_coro=demo_exc_handling()
next(exc_coro)

->continue started


In [21]:
exc_coro.send(11)

->coroutine received: 11


In [22]:
exc_coro.throw(DemoException)#异常得到了处理，协程不会终止

***DemoException handled.Continue ...


In [23]:
getgeneratorstate(exc_coro)

'GEN_SUSPENDED'

In [24]:
#如果传入没法处理的异常，协程会终止
exc_coro=demo_exc_handling()
next(exc_coro)

->continue started


In [25]:
exc_coro.throw(ZeroDivisionError)

ZeroDivisionError: 

In [26]:
getgeneratorstate(exc_coro)

'GEN_CLOSED'

In [27]:
#如果不过协程如何执行都想做些清理工作，要把协程定义体中相关的代码放入try/finallu块中
def demo_finally():
    print('->continue started')
    try:
        while True:
            try:
                x=yield 
            except DemoException:
                print('***DemoException handled.Continue ...')
            else:
                print('->coroutine received: {!r}'.format(x))
    finally:print('->coroutine ending')

In [28]:
exc_coro=demo_finally()
next(exc_coro)

->continue started


In [29]:
next(exc_coro)

->coroutine received: None


In [30]:
exc_coro.send(55)

->coroutine received: 55


In [31]:
exc_coro.throw(ZeroDivisionError)

->coroutine ending


ZeroDivisionError: 

16.6 让协程返回值

In [33]:
from collections import namedtuple

Result = namedtuple('Result', 'count average')


def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield
        if term is None:#为了返回值，协程必须终止
            break  # <1>
        total += term
        count += 1
        average = total/count
    return Result(count, average)  # <2>
# END RETURNING_AVERAGER

In [35]:
coro_avg=averager()
next(coro_avg)#不产出值

In [36]:
coro_avg.send(10)#不产出值

In [37]:
coro_avg.send(30)

In [38]:
coro_avg.send(6.5)

In [39]:
coro_avg.send(None)#终止循环，导致协程结束，返回结果，异常对象的value属性保存着返回值

StopIteration: Result(count=3, average=15.5)

In [40]:
#不会StopIteration异常，获取averager返回的值
coro_avg=averager()
next(coro_avg)
coro_avg.send(10)

In [41]:
coro_avg.send(30)

In [42]:
coro_avg.send(6.5)

In [43]:
try:
    coro_avg.send(None)
except StopIteration as exc:
    result=exc.value
    

In [44]:
result

Result(count=3, average=15.5)

16.7 使用yield from

In [45]:
def gen():
    for c in 'AB':
        yield c
    for i in range(1,3):
        yield i

In [46]:
list(gen())

['A', 'B', 1, 2]

In [47]:
#可以改写为：
def gen():
    yield from 'AB'
    yield from  range(1,3)

In [48]:
list(gen())

['A', 'B', 1, 2]

yield from x表达式对x对象所做的第一件事，调用iter(x)，从中获取迭代器
yield from的主要功能是打开双向通道，把最外层的调用方与最内层的子生成器连接起来，这样两者可以直接发送和产出值，还可以直接传入异常，而不用在位于中间的协程中大量处理异常的样板代码

![chapter16-7](image/chapter16-7.jpg)

In [28]:
#用yield from计算平均值并输出统计报告
from collections import namedtuple

Result = namedtuple('Result', 'count average')

#子生成器
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average#main函数中的客户代码发送的各个值绑定到这里的term变量上
        if term is None:#至关重要的终止条件，若不这样做，使用yield from调用这个协程的生成器会永远阻塞
            break  # <1>
        total += term
        count += 1
        average = total/count
    return Result(count, average)  # 返回的Result会成为grouper函数中yield from表达式的值
# END RETURNING_AVERAGER

#委派生成器
def grouper(results,key):
    while True:#每次迭代都会新建一个averager实例，每个实例都是昨晚协程使用的生成器
        results[key]=yield from averager()#grouper发送的每个值都会经过yield from处理，通过管道传给averager实例
                                            #averager实例运行完后，返回的值绑定到resuls[key]上
                                        #对yield from来说，解释器不仅会捕获StopIteration异常，还会把value的属性的值变
                                        # 成yield from表达式的值

#客户端代码，即调用方
def main(data):
    results={}
    for key,values in data.items():
        group=grouper(results,key)
        next(group)
        for value in values:
            group.send(value)
        group.send(None)
    report(results)
    print(results)
    
# output report
def report(results):
    for key, result in sorted(results.items()):
        group, unit = key.split(';')
        print('{:2} {:5} averaging {:.2f}{}'.format(
              result.count, group, result.average, unit))


data = {
    'girls;kg':
        [40.9, 38.5, 44.3, 42.2, 45.2, 41.7, 44.5, 38.0, 40.6, 44.5],
    'girls;m':
        [1.6, 1.51, 1.4, 1.3, 1.41, 1.39, 1.33, 1.46, 1.45, 1.43],
    'boys;kg':
        [39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
    'boys;m':
        [1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46],
}


if __name__ == '__main__':
    main(data)

 9 boys  averaging 40.42kg
 9 boys  averaging 1.39m
10 girls averaging 42.04kg
10 girls averaging 1.43m
{'girls;kg': Result(count=10, average=42.040000000000006), 'girls;m': Result(count=10, average=1.4279999999999997), 'boys;m': Result(count=9, average=1.3888888888888888), 'boys;kg': Result(count=9, average=40.422222222222224)}


In [21]:
ree={}
group=grouper(ree,'kry')#grouper是一个生成器

In [22]:
next(group)

In [23]:
group.send(2)

2.0

In [24]:
group.send(3)

2.5

In [25]:
group.send(None)

In [26]:
ree

{'kry': Result(count=2, average=2.5)}

In [29]:
results={}
for key,values in data.items():
    avg=averager()
    next(avg)
    for value in values:
        avg.send(value)
    results[key]=avg.send(None)
results
    
        

StopIteration: Result(count=10, average=42.040000000000006)

综上，yield from的主要功能是打开双向通道，把最外层的调用方与最内层的子生成器连接起来，这样二者可以直接发送和产出值，还可以直接传入异常，而不用在位于中间的协程中添加大量处理异常的样板代码。

对yield from来说，解释器不仅会捕获StopIteration异常，还会把value的属性的值变 成yield from表达式的值